In [ ]:
#librerías
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.layers import Dense
import pickle



# Leer los datos 

# PP_SEGMENTADO_ABC

In [ ]:
df = pd.read_csv('PP_Segmentado_ABC.csv')

In [ ]:
df.head()

In [ ]:
columns_m0 = ['Saldo_total', 'Saldo_Mes', 'Pago_minimo_M0', 'Utilizacion', 'Pago_M0', 'Variable_objetivo', 'Variable_Objetivo_m1', 'L2_Pago', 'dias_deudados_m0', 'Canal_Pago_M1','Canal_Pago_M2','Canal_Pago_M3','Canal_Pago_M4','Canal_Pago_M5','Canal_Pago_M6']

df_0 = df.drop(columns = columns_m0).copy()

In [ ]:
df_0.columns

In [ ]:
df_1 = pd.get_dummies(df_0, drop_first=True)

In [ ]:
df_1.columns

In [ ]:
# df_1.info()

In [ ]:
df_2 = df_1[df_1['Saldo_total_M1'].notna()].fillna(0)

In [ ]:
# df_2.info()

In [ ]:
df_2.columns

*Red sencilla*

In [ ]:
# Ver columnas con datos no numéricos
print(df.dtypes)


In [ ]:
print(df_2.shape)  # (n_filas, n_columnas)


In [ ]:
print(df_2['Pago_M1'].describe())


In [ ]:
print(df_2.dtypes)


# Mejor modelo para PP_Segmentado

In [ ]:
# 1. Fijar la semilla para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Separar features y target
X = df_2.drop(['Pago_M1'], axis=1).values.astype('float32')  # aseguramos float
y = df_2['Pago_M1'].values.astype('float32')

# 3. Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Dividir en entrenamiento y validación con split aleatorio reproducible
n_samples = X_scaled.shape[0]
indices = np.random.permutation(n_samples)
split = int(0.8 * n_samples)
train_idx, val_idx = indices[:split], indices[split:]

X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
y_train, y_val = y[train_idx], y[val_idx]

# 5. Crear el modelo
model_pp = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1)  # Salida para regresión
])

# 6. Compilar el modelo
model_pp.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 7. Entrenar el modelo
model_pp.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val))

# 8. Evaluar el modelo
loss, mae = model_pp.evaluate(X_val, y_val)
print(f'MAE en validación: {mae:.2f}')


In [ ]:
# Guardar el modelo entrenado
with open('modelo_entrenado_pp.pkl', 'wb') as f:
    pickle.dump(model_pp, f)


##Para cargar el archivo
##with open('modelo_entrenado_pp.pkl', 'rb') as f:
  ##  resultados_cargados_pp = pickle.load(f)

##print(resultados_cargados)

In [ ]:
y_pred = model_pp.predict(X_val).flatten()

mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print("R2:", r2_score(y_val, y_pred))
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

### K-Fold Cross-Validation

In [ ]:

# Fijar semilla
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Convertir booleanos a float
df_2 = df_2.astype({col: 'float' for col in df_2.select_dtypes('bool').columns})

# Preparar datos
X = df_2.drop(['Pago_M1'], axis=1).values.astype('float32')
y = df_2['Pago_M1'].values.astype('float32')

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Listas para métricas
mae_list = []
mse_list = []
r2_list = []

# Función para crear modelo (necesario para reiniciar en cada fold)
def build_model_pp(input_dim):
    model_pp = Sequential([
        Dense(32, activation='relu', input_shape=(input_dim,)),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1)  # Salida regresión
    ])
    model_pp.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model_pp

# K-Fold
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled), 1):
    print(f"\n Fold {fold}")

    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    model_pp = build_model_pp(X_train.shape[1])
    model_pp.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)

    y_pred = model_pp.predict(X_test).flatten()  # Aplanar para métricas de sklearn
    
    mae_list.append(mean_absolute_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))
    r2_list.append(r2_score(y_test, y_pred))

# Resultados
print("\n📊 Resultados por fold:")
print("MAE por fold:", mae_list)
print("MSE por fold:", mse_list)
print("R² por fold:", r2_list)


### Malo

In [ ]:
registro_n = 1413
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_pp.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")


### Regular

In [ ]:
registro_n = 234
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_pp.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")


### Bueno

In [ ]:
registro_n = 582
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_pp.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")

# TDC_DEPT_SEGMENTADO_ABC

In [ ]:
df_TDC_D = pd.read_csv('TDC_Dept_Segmentado_ABC.csv')


In [ ]:
df_TDC_D.head()


In [ ]:
columns_m0 = ['Saldo_total', 'Saldo_Mes', 'Pago_minimo_M0', 'Utilizacion', 'Pago_M0', 'Variable_objetivo', 'Variable_Objetivo_m1', 'L2_Pago', 'dias_deudados_m0', 'Canal_Pago_M1','Canal_Pago_M2','Canal_Pago_M3','Canal_Pago_M4','Canal_Pago_M5','Canal_Pago_M6']

df_0 = df_TDC_D.drop(columns = columns_m0).copy()

In [ ]:
df_1 = pd.get_dummies(df_0, drop_first=True)

In [ ]:
df_2_TDC_D = df_1[df_1['Saldo_total_M1'].notna()].fillna(0)

# Mejor modelo para TDC_Dept_Segmentado 

In [ ]:
# 1. Fijar la semilla para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


# Separar features y target
X = df_2_TDC_D.drop(['Pago_M1'], axis=1).values.astype('float32')  # aseguramos float
y = df_2_TDC_D['Pago_M1'].values.astype('float32')

# 3. Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Dividir en entrenamiento y validación con split aleatorio reproducible
n_samples = X_scaled.shape[0]
indices = np.random.permutation(n_samples)
split = int(0.8 * n_samples)
train_idx, val_idx = indices[:split], indices[split:]

X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
y_train, y_val = y[train_idx], y[val_idx]

# 5. Crear el modelo
model_tdc_d = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1)  # Salida para regresión
])

# 6. Compilar el modelo
model_tdc_d.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 7. Entrenar el modelo
model_tdc_d.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val))

# 8. Evaluar el modelo
loss, mae = model_tdc_d.evaluate(X_val, y_val)
print(f'MAE en validación: {mae:.2f}')
y_pred = model_tdc_d.predict(X_val).flatten()
print("R2:", r2_score(y_val, y_pred))
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


In [ ]:
# Guardar el modelo entrenado
with open('modelo_entrenado_tdc_d.pkl', 'wb') as f:
    pickle.dump(model_tdc_d, f)


##Para cargar el archivo
##with open('modelo_entrenado_tdc_d.pkl', 'rb') as f:
  ##  resultados_cargados_tdc_d = pickle.load(f)

##print(resultados_cargados)

### K-Fold Cross-Validation

In [ ]:
# Fijar semilla
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Convertir booleanos a float
df_2_TDC_D = df_2_TDC_D.astype({col: 'float' for col in df_2.select_dtypes('bool').columns})

# Preparar datos
X = df_2_TDC_D.drop(['Pago_M1'], axis=1).values.astype('float32')
y = df_2_TDC_D['Pago_M1'].values.astype('float32')

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Listas para métricas
mae_list = []
mse_list = []
r2_list = []

# Función para crear modelo (necesario para reiniciar en cada fold)
def build_model_tdc_d(input_dim):
    model_tdc_d = Sequential([
        Dense(32, activation='relu', input_shape=(input_dim,)),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1)  # Salida regresión
    ])
    model_tdc_d.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model_tdc_d

# K-Fold
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled), 1):
    print(f"\n📂 Fold {fold}")

    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    model_tdc_d = build_model_tdc_d(X_train.shape[1])
    model_tdc_d.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)

    y_pred = model_tdc_d.predict(X_test).flatten()  # Aplanar para métricas de sklearn
    
    mae_list.append(mean_absolute_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))
    r2_list.append(r2_score(y_test, y_pred))

# Resultados
print("\n📊 Resultados por fold:")
print("MAE por fold:", mae_list)
print("MSE por fold:", mse_list)
print("R² por fold:", r2_list)


### Malo

In [ ]:
registro_n = 83
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_tdc_d.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")


### Regular

In [ ]:
registro_n = 1562
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_tdc_d.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")


### Bueno

In [ ]:
registro_n = 1179
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_tdc_d.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")

# TDC Visa Segmentado ABC

In [ ]:
df_TDC_V = pd.read_csv('TDC_Visa_Segmentado_ABC.csv')


In [ ]:
df_TDC_V.head()


In [ ]:
columns_m0 = ['Saldo_total', 'Saldo_Mes', 'Pago_minimo_M0', 'Utilizacion', 'Pago_M0', 'Variable_objetivo', 'Variable_Objetivo_m1', 'L2_Pago', 'dias_deudados_m0', 'Canal_Pago_M1','Canal_Pago_M2','Canal_Pago_M3','Canal_Pago_M4','Canal_Pago_M5','Canal_Pago_M6']

df_0 = df_TDC_V.drop(columns = columns_m0).copy()

In [ ]:
df_1 = pd.get_dummies(df_0, drop_first=True)

In [ ]:
df_2_TDC_V = df_1[df_1['Saldo_total_M1'].notna()].fillna(0)

# Mejor modelo MLP para TDC Visa segmentado 

In [ ]:
# 1. Fijar la semilla para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

X = df_2_TDC_V.drop(['Pago_M1'], axis=1).values
y = df_2_TDC_V['Pago_M1'].values

# 3. Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Dividir en entrenamiento y validación con split aleatorio reproducible
n_samples = X_scaled.shape[0]
indices = np.random.permutation(n_samples)
split = int(0.8 * n_samples)
train_idx, val_idx = indices[:split], indices[split:]

X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
y_train, y_val = y[train_idx], y[val_idx]

# 5. Crear el modelo
model_TDC_V = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1)  # Salida para regresión
])

# 6. Compilar el modelo
model_TDC_V.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 7. Entrenar el modelo
model_TDC_V.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val))

# 8. Evaluar el modelo
loss, mae = model_TDC_V.evaluate(X_val, y_val)
print(f'MAE en validación: {mae:.2f}')
y_pred = model_TDC_V.predict(X_val).flatten()

mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print("R2:", r2_score(y_val, y_pred))

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


In [ ]:
import pickle

# Guardar el modelo entrenado
with open('modelo_entrenado_tdc_v.pkl', 'wb') as f:
    pickle.dump(model_TDC_V, f)


##Para cargar el archivo
##with open('modelo_entrenado_tdc_v.pkl', 'rb') as f:
  ##  resultados_cargados_tdc_v = pickle.load(f)

##print(resultados_cargados)


### K-Fold Cross-Validation

In [ ]:

# Fijar semilla
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Convertir booleanos a float
df_2_TDC_V = df_2_TDC_V.astype({col: 'float' for col in df_2.select_dtypes('bool').columns})


# Preparar datos
X = df_2_TDC_V.drop(['Pago_M1'], axis=1).values.astype('float32')
y = df_2_TDC_V['Pago_M1'].values.astype('float32')

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Listas para métricas
mae_list = []
mse_list = []
r2_list = []

# Función para crear modelo (necesario para reiniciar en cada fold)
def build_model_tdc_v(input_dim):
    model_tdc_v = Sequential([
        Dense(32, activation='relu', input_shape=(input_dim,)),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1)  # Salida regresión
    ])
    model_tdc_v.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model_tdc_v

# K-Fold
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled), 1):
    print(f"\n📂 Fold {fold}")

    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    model_tdc_v = build_model_tdc_v(X_train.shape[1])
    model_tdc_v.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)

    y_pred = model_tdc_v.predict(X_test).flatten()  # Aplanar para métricas de sklearn
    
    mae_list.append(mean_absolute_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))
    r2_list.append(r2_score(y_test, y_pred))

# Resultados
print("\n📊 Resultados por fold:")
print("MAE por fold:", mae_list)
print("MSE por fold:", mse_list)
print("R² por fold:", r2_list)


### Malo

In [ ]:
registro_n = 122
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_tdc_v.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")

### Regular

In [ ]:
registro_n = 1382
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_tdc_v.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")

### Bueno

In [ ]:
registro_n = 128
punto_original = X[registro_n].reshape(1, -1) 

# Escalar el punto individual
punto_escalado = scaler.transform(punto_original)

# Usar el último modelo del último fold
prediccion = model_tdc_v.predict(punto_escalado).flatten()[0]

print(f"Predicción del modelo para el registro {registro_n}: {prediccion:.2f}")